# Light Gradient Boosting 
an Highly efficient Gradient Boosting Tree Algorithm

Summarized by QH 

_first draft: 2022-12-02_

## Advantages of LightGBM
* Optimization in Speed and Memory Usage
    * Many boosting tools use _pre-sort-based algorithm_ (by default for xgboost) for decision tree learning: simple solution but not easy to optimize. LightGBM uses _histogram-based algorithm_ (bucketing continuous feature values into discrete bins). It speeds up training and reduces memory usage:
        * Reduced cost of calculating the gain for each split
            * Pre-sort-based algorithms have time complexity $O(n)$, $n$ is the number of observations for data.
            * Computing the histogram has time complexity $O(n)$, but this involves only a fast sum-up operation. Once the histogram is constructed, a histogram-based algorithm has time complexity $O(b)$, $b$ is the number of bins, and $b$ is far smaller than $n$.
        * Use histogram subtraction for further speedup
            * To get one leaf’s histograms in a binary tree, use the histogram subtraction of its parent and its neighbor. So it needs to construct histograms for only one leaf (with smaller $n$ than its neighbor). It then can get histograms of its neighbor by histogram subtraction with small cost ($O(b)$)
        * Reduce memory usage
            * Replaces continuous values with discrete bins. If $b$ is small, can use small data type to store training data
            * No need to store additional information for pre-sorting feature values
        * Reduce communication cost for distributed learning
* Optimization in Accuracy
    * Leaf-wise (Best-first) Tree growth
        * Most decision tree learning algorithm grow trees by level:
        
        <img src="level-wise.png" alt="level-wise_png" width="350"/>
        
        * LightGBM grows trees leaf-wise which chooses the leaf with max delta loss to grow. Holding #leaf fixed, leaf-wise algorithms tend to achieve lower loss than level-wise algorithms. Though the leaf-wise may cause over-fitting when #data is small, `max_depth` parameter can help to limit the tree depth to control the over-fitting.

        <img src="leaf-wise.png" alt="leaf-wise_png" width="400"/>
    * Optimal Split for Categorical Features
    * Optimization in Distributed Learning

## Parameters (v3.3.2)
| Parameters | Description | Possible values |
| :--|:-- | :-- |
|`num_leaves` (default = 30)|Maximum tree leaves for base learners.|_int_ $[0, \infty)$|
|`max_depth` (default=-1)|Maximum depth of a tree. Increasing this value will make the model more complex and more likely to overfit.|_int_:df $[0,\infty)$|
|`learning_rate` (default = 0.1)|Step size shrinkage used in update to prevents overfitting.|range: $[0, 1]$|
|`n_estimators` (default = 100)|Number of gradient boosted trees. Equivalent to number of boosting rounds||
|`subsample_for_bin` (default = 200000|Number of samples for constructing bins|_int_|
|`objective`|Specify the learning task and the corresponding learning objective or a custom objective function to be used.|'regression' for LGBMRegressor, 'binary' or 'multiclass' for LGBMClassifier, 'lambdarank' for LGBMRanker|
|`class_weight` (default=None)|Weights associated with classes in the form {class_label: weight}. Use this parameter only for multi-class classification task; for binary classification task you may use `is_unbalance` or `scale_pos_weight` parameters.||
|`min_split_gain` (default = 0)|Minimum loss reduction required to make a further partition on a leaf node of the tree.|range: $[0,\infty)$|
|`min_child_weight` (default = 1e^-3)|Minimum sum of instance weight (hessian) needed in a child.The larger `min_child_weight` is, the more conservative the algorithm will be|range: $[0,\infty)$|
|`min_child_samples` (default = 20)|Minimum number of data needed in a child (leaf).|range: $[0,\infty)$|
|`subsample`(default = 1)|Subsample ratio of the training instances. Setting it to 0.5 means that LightGBM would randomly sample half of the training data prior to growing trees.|range: $(0, 1]$|
|`colsample_bytree`(default = 1)|subsample ratio of columns when constructing each tree. Subsampling occurs once for every tree constructed|range: $(0, 1]$|
|`reg_lambda`(default = 1)|L2 regularization term on weights. Increasing this value will make model more conservative.|range: $[0,\infty)$|
|`reg_alpha`(default = 0)|L1 regularization term on weights. Increasing this value will make model more conservative.|-|
|`random_state`|Random number seed.||
|`n_jobs`(default = -1)|Number of parallel threads.|-|
|`silent` (default = True)|Whether to print messages while running boosting.||

## References
1. https://lightgbm.readthedocs.io/en/v3.3.2/Features.html